# Find Nearby Commercial Center

In [28]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from sklearn import preprocessing
from sklearn.neighbors import DistanceMetric
from math import radians
%matplotlib inline

checking_distance = 1
df_train = pd.read_csv("project_data/train.csv")
commercial_centers = pd.read_csv("project_data/auxiliary-data/sg-commerical-centres.csv")
df_train['radian_latitude']= np.radians(df_train['latitude'])
df_train['radian_longitude'] = np.radians(df_train['longitude'])

commercial_centers['radian_lat']= np.radians(commercial_centers['lat'])
commercial_centers['radian_lng'] = np.radians(commercial_centers['lng'])

# The distance computed here is a haversine distance. 
# This assumes the earth is a true sphere which makes for a relatively fast computation. 
# Reference: https://medium.com/@danalindquist/finding-the-distance-between-two-lists-of-geographic-coordinates-9ace7e43bb2f

dist = DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (df_train[['radian_latitude','radian_longitude']],
     commercial_centers[['radian_lat','radian_lng']])*6378
)
# the radius of the earth in miles: 3959 
# the radius of the earth in km: 6378 km
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=df_train['street_name'], 
                 columns=commercial_centers['name'])
)

# Finding the nearby commercial centers which are less than 1km distance
df_dist_matrix['Count_Commercial']=df_dist_matrix.iloc[::].le(checking_distance,axis = 0).sum(axis = 1)
df_dist_matrix=df_dist_matrix[['Count_Commercial']]

df_train_commercial = pd.DataFrame(
    np.column_stack([df_train,df_dist_matrix]),
    columns=df_train.columns.append(df_dist_matrix.columns)
)
pd.set_option('display.max_columns', None)
df_train_commercial.head(20)



,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region,resale_price,radian_latitude,radian_longitude,Count_Commercial
0,2001-08,pasir ris,4 room,440,pasir ris drive 4,01 to 03,118.0,model a,uncategorized,1989,1.369008,103.958697,0.0,pasir ris drive,pasir ris,east region,209700.0,0.023894,1.814422,0
1,2014-10,punggol,5-room,196B,punggol field,10 to 12,110.0,improved,uncategorized,2003,1.399007,103.906991,0.0,punggol field,punggol,north-east region,402300.0,0.024417,1.813519,0
2,2020-09,sengkang,5 room,404A,fernvale lane,01 to 03,112.0,premium apartment,uncategorized,2004,1.388348,103.873815,0.0,fernvale,sengkang,north-east region,351000.0,0.024231,1.81294,0
3,2000-10,clementi,3 room,375,clementi avenue 4,07 to 09,67.0,new generation,uncategorized,1980,1.318493,103.766702,0.0,clementi north,clementi,west region,151200.0,0.023012,1.811071,0
4,2013-01,bukit batok,3-room,163,bukit batok street 11,07 to 09,73.0,model a,uncategorized,1985,1.348149,103.742658,0.0,bukit batok west,bukit batok,west region,318600.0,0.02353,1.810651,0
5,2001-01,bukit batok,3 room,240,Bukit Batok East Avenue 5,10 to 12,67.0,new generation,uncategorized,1985,1.350488,103.754178,0.0,bukit batok east,bukit batok,west region,120600.0,0.02357,1.810852,0
6,2007-09,jurong west,4 room,647,jurong west street 61,13 to 15,100.0,model a,uncategorized,1998,1.340111,103.697388,0.0,jurong west central,jurong west,west region,248850.0,0.023389,1.809861,0
7,2010-11,geylang,5-room,3,pine close,16 to 18,110.0,improved,uncategorized,2000,1.306929,103.88311,0.0,aljunied,geylang,central region,639000.0,0.02281,1.813102,0
8,2005-08,yishun,3 room,239,yishun ring road,04 to 06,67.0,new generation,uncategorized,1985,1.433595,103.839685,0.0,northland,yishun,north region,136800.0,0.025021,1.812344,0
9,2013-10,bukit batok,3-room,504,bukit batok street 52,10 to 12,60.0,improved,uncategorized,1988,1.351966,103.754984,0.0,guilin,bukit batok,west region,261000.0,0.023596,1.810866,0


In [36]:
pd.set_option('display.max_rows', None)
hawker_centers = pd.read_csv("project_data/auxiliary-data/sg-gov-markets-hawker-centres.csv")
hawker_centers['radian_lat']= np.radians(commercial_centers['lat'])
print(hawker_centers)
hawker_centers['radian_lng'] = np.radians(commercial_centers['lng'])
# print(hawker_centers)

# The distance computed here is a haversine distance. 
# This assumes the earth is a true sphere which makes for a relatively fast computation. 
# Reference: https://medium.com/@danalindquist/finding-the-distance-between-two-lists-of-geographic-coordinates-9ace7e43bb2f

dist = DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (df_train_commercial[['radian_latitude','radian_longitude']],
     hawker_centers[['radian_lat','radian_lng']])*6378
)
# the radius of the earth in miles: 3959 
# the radius of the earth in km: 6378 km
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=df_train_commercial['street_name'], 
                 columns=hawker_centers['name'])
)
# print(df_dist_matrix.head(20))

# Finding the nearby hawker centers which are less than 1km distance
df_dist_matrix['Count_Hawkers']=df_dist_matrix.iloc[::].le(checking_distance,axis = 0).sum(axis = 1)

df_train_commercial_hawker = pd.DataFrame(
    np.column_stack([df_train_commercial,df_dist_matrix]),
    columns=df_train_commercial.columns.append(df_dist_matrix.columns)
)
pd.set_option('display.max_columns', None)
# df_train_commercial_hawker.head(20)



                                                  name       lat         lng  \
0                       East Coast Lagoon Food Village  1.307730  103.934303   
1                            Jurong West Hawker Centre  1.341223  103.697374   
2    Bukit Merah View Blk 115 (Blk 115 Bukit Merah ...  1.285240  103.822372   
3    Geylang Bahru Blk 69 (Blk 69 Geylang Bahru Mar...  1.321530  103.870071   
4                                 Ghim Moh Road Blk 20  1.311060  103.788292   
5    Telok Blangah Crescent Blk 11 (11 Telok Blanga...  1.277440  103.818733   
6                            Toa Payoh Lorong 4 Blk 93  1.338540  103.849571   
7                            Toa Payoh Lorong 5 Blk 75  1.336100  103.852997   
8                           Toa Payoh Lorong 8 Blk 210  1.340330  103.854462   
9    Upper Boon Keng Road Blk 17 (Blk 17 Upper Boon...  1.315090  103.871681   
10   New Market Road Blk 32 (People's Park Food Cen...  1.284871  103.842590   
11                        New Upper Chan